In [987]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
# import requests
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
# pip install xlsxwriter
# pip install xlwx

In [988]:
mergedData = pd.read_csv("merged_eviction_data.csv")

In [989]:
fundingSource = mergedData.drop(columns = ['Administering Agency','Max $ Assistance per household (total)',
       'Household Assistance Details (# of months, max per month, etc)',
       'Area Median Income Elgibility', 'Program Timeline', 'Contact info',
       'Link to application', 'Notes', 'Unnamed: 11', 'Residents 2020-10-24',
       'Count 2020-10-24', 'Residents 2020-10-31', 'Count 2020-10-31',
       'Residents 2020-11-07', 'Count 2020-11-07', 'Residents 2020-11-14',
       'Count 2020-11-14', 'Residents 2020-11-21', 'Count 2020-11-21',
       'Residents 2020-11-28', 'Count 2020-11-28', 'Residents 2020-12-05',
       'Count 2020-12-05', 'Residents 2020-12-12', 'Count 2020-12-12',
       'Residents 2020-12-19', 'Count 2020-12-19', 'Residents 2020-12-26',
       'Count 2020-12-26', 'Residents 2021-01-02', 'Count 2021-01-02',
       'Residents 2021-01-09', 'Count 2021-01-09', 'Residents 2021-01-16',
       'Count 2021-01-16', 'Residents 2021-01-23', 'Count 2021-01-23',
       'Residents 2021-01-30', 'Count 2021-01-30', 'Residents 2021-02-06',
       'Count 2021-02-06', 'Residents 2021-02-13', 'Count 2021-02-13',
       'Residents 2021-02-20', 'Count 2021-02-20', 'Residents 2021-02-27',
       'Count 2021-02-27'])

In [990]:
# fundingSource = fundingSource.replace({'Funding Source' : {'Trust/CPA' : 'Trust, CPA', ', ' : ';','; ' : ';','and':''}})

fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(', and ', '/')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(', ', '/')
# fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(' / ', '/')
# # fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace(' /', '/')

fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust and CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust, CPA', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CPA/Trust', 'Trust/CPA')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('Trust and CDBG', 'Trust/CDBG')

fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CARES Act funds', 'CARES Act')
fundingSource['Funding Source'] = fundingSource['Funding Source'].str.replace('CARES Act ', 'CARES Act')

In [991]:
fundingSource.to_csv('fundingsources.csv')

In [992]:
# CARES act funding only
cares = pd.DataFrame()
cares = fundingSource[fundingSource['Funding Source'] == 'CARES Act']
cares = cares.rename(columns={'Amount Allocated':'CARES'})
cares = cares.drop(columns=['Funding Source'])

In [993]:
trust = pd.DataFrame()
trust = fundingSource[fundingSource['Funding Source'] == 'Trust']
trust = trust.rename(columns={'Amount Allocated':'Trust'})
trust = trust.drop(columns=['Funding Source'])

In [994]:
notSpecified = pd.DataFrame()
notSpecified = fundingSource[fundingSource['Funding Source'] == 'Not specified']
notSpecified = notSpecified.rename(columns={'Amount Allocated':'Not Specified'})
notSpecified = notSpecified.drop(columns=['Funding Source'])

In [995]:
trustCPA = pd.DataFrame()
trustCPA = fundingSource[fundingSource['Funding Source'] == 'Trust/CPA']
trustCPA = trustCPA.rename(columns={'Amount Allocated':'Trust/CPA'})
trustCPA = trustCPA.drop(columns=['Funding Source'])

In [996]:
## fixing some of the municipalities that have multiple sources

## ARLINGTON
values = fundingSource.loc[fundingSource['Municipality'] == 'Arlington',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()[1].split(', ')
a = dict([('Municipality','Arlington'),('CPA',values[1]),('CDBG-CV',values[0]),('Donations',values[2]),('Housing Corporation',values[3])])
arlington = pd.DataFrame.from_dict(a,orient='index').transpose()

## BROOKLINE
values = fundingSource.loc[fundingSource['Municipality'] == 'Brookline',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
values
b = dict([('Municipality','Brookline'),('CDBG-CV',values[2].split('and')[1]),('Town Sources', values[1]),('Housing Trust',values[2].split('and')[0]), ('Private Fundraising', values[2].split('and')[2])])
brookline = pd.DataFrame.from_dict(b,orient='index').transpose()


## BOSTON
values = fundingSource.loc[fundingSource['Municipality'] == 'Boston',['Municipality','Amount Allocated','Funding Source']].values.flatten().tolist()
bos = dict([('Municipality','Boston'),('Rental Relief Lottery', values[2].split(' - ')[0]),('Federal Stimulus Funds',values[2].split(' - ')[1])])
boston = pd.DataFrame.from_dict(bos,orient='index').transpose()
boston

,Municipality,Rental Relief Lottery,Federal Stimulus Funds
0,Boston,Round 1 of a rental relief lottery occurred on...,$5 million in federal stimulus funds.


In [997]:
extras = pd.merge(arlington,brookline,how='outer')
extras = pd.merge(extras,boston,how='outer')
extras

,Municipality,CPA,CDBG-CV,Donations,Housing Corporation,Town Sources,Housing Trust,Private Fundraising,Rental Relief Lottery,Federal Stimulus Funds
0,Arlington,"$300,000 CPA (starting late summer/early fall","$400,000 CDBG-CV funds","$115,000 in donations through Arlington Health...","and $100,000 through Housing Corporation of Ar...",NaN,NaN,NaN,NaN,NaN
1,Brookline,NaN,"$375,000 from CDBG",NaN,NaN,$1.2 million from Town Sources plus Private Fu...,"$100,000 from Brookline Housing Trust","$725,000 from Private Fundraising",NaN,NaN
2,Boston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Round 1 of a rental relief lottery occurred on...,$5 million in federal stimulus funds.


In [998]:
cpa = pd.DataFrame()
cpa = fundingSource[fundingSource['Funding Source'] == 'CPA']
cpa = cpa.rename(columns={'Amount Allocated':'CPA'})
cpa = cpa.drop(columns=['Funding Source'])

In [999]:
esgCv = pd.DataFrame()
esgCv = fundingSource[fundingSource['Funding Source'] == 'ESG-CV']
esgCv = esgCv.rename(columns={'Amount Allocated':'ESG-CV'})
esgCv = esgCv.drop(columns=['Funding Source'])

In [1000]:
home = pd.DataFrame()
home = fundingSource[fundingSource['Funding Source']=='HOME']
home = home.rename(columns={'Amount Allocated':'HOME'})
home = home.drop(columns=['Funding Source'])

In [1001]:
cdbgHome = pd.DataFrame()
cdbgHome = fundingSource[fundingSource['Funding Source']=='CDBG/HOME-TBRA']
cdbgHome = cdbgHome.rename(columns={'Amount Allocated':'CDBG/HOME-TBRA'})
cdbgHome = cdbgHome.drop(columns=['Funding Source'])

In [1002]:
city = pd.DataFrame()
city = fundingSource[fundingSource['Funding Source']=='City funds allocated by Mayor']
city = city.rename(columns={'Amount Allocated':'City Funds Allocated by Mayor'})
city = city.drop(columns=['Funding Source'])

In [1005]:
cambridge = pd.DataFrame()
cambridge = fundingSource[fundingSource['Funding Source']=="Donations were used for Mayor's Distaster Relief Funds/"]
cambridge = cambridge.rename(columns={'Amount Allocated':"Donations were used for Mayor's Distaster Relief Funds"})
cambridge = cambridge.drop(columns=['Funding Source'])

In [1006]:
trustCdbgCv = pd.DataFrame()
trustCdbgCv = fundingSource[fundingSource['Funding Source']=="Trust/CDBG-CV"]
trustCdbgCv = trustCdbgCv.rename(columns={'Amount Allocated':"Trust/CDBG-CV"})
trustCdbgCv = trustCdbgCv.drop(columns=['Funding Source'])

In [1007]:
sourcesTrack = pd.DataFrame(columns=['Municipality'])
sourcesTrack['Municipality'] = fundingSource['Municipality']
sourcesTrack = pd.merge(sourcesTrack, cares, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, trust, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, notSpecified, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, trustCPA, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, cpa, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, esgCv, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, home, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, cdbgHome, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, city, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, cambridge, how='left',on='Municipality')
sourcesTrack = pd.merge(sourcesTrack, trustCdbgCv, how='left',on='Municipality')


sourcesTrack.to_csv('sourceTracker.csv')
sourcesTrack

,Municipality,CARES,Trust,Not Specified,Trust/CPA,CPA,ESG-CV,HOME,CDBG/HOME-TBRA,City Funds Allocated by Mayor,Donations were used for Mayor's Distaster Relief Funds,Trust/CDBG-CV
0,Acton,"$55,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Amesbury,NaN,NaN,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Amherst,NaN,"$250,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barnstable County,NaN,NaN,NaN,NaN,NaN,NaN,"$100,000",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
82,Weymouth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Williamstown,NaN,NaN,NaN,"$20,700 Trust, $75,000 CPA",NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,Yarmouth,NaN,NaN,NaN,"$108,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,"South Berkshire Consortium, see notes*",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1008]:
sourcesTrack[sourcesTrack['Municipality'] == 'Brockton']['HOME','CDGB-CV']= sourcesTrack

,Municipality,CARES,Trust,Not Specified,Trust/CPA,CPA,ESG-CV,HOME,CDBG/HOME-TBRA,City Funds Allocated by Mayor,Donations were used for Mayor's Distaster Relief Funds,Trust/CDBG-CV
10,Brockton,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$250,000 CDBG, $250,000 HOME-TBRA",NaN,NaN,NaN
